### Load Images by ver4

In [2]:
from sklearn.model_selection import train_test_split
from glob import glob
import os
import shutil
from collections import defaultdict
from datetime import datetime

In [3]:
VOCROOTDIR = "../data/2022_10"
img_list = []
files = os.listdir(VOCROOTDIR + "/" + str(0) + "/" )
img_files = list(filter(lambda x: '.jpg' in x, files))
img_list += img_files

In [4]:
image_types = {"IMG":defaultdict(list), "PANO":defaultdict(list), "ETC":[]}

In [5]:
for file_name in img_list:
    _split = file_name.split("_")
    if 2 < len(_split):
        image_types[_split[0]][_split[1]].append(file_name)
    else:
        image_types["ETC"].append(file_name)

---
### Imgae Compare

In [13]:
from skimage.metrics import structural_similarity as ssim 
import argparse
import cv2
from skimage import io
from itertools import combinations
import matplotlib.pyplot as plt

from PIL import Image, ImageFont, ImageDraw

from torchvision.transforms.functional import to_pil_image
from tqdm import tqdm

In [14]:
def get_concat_h(im1, im2,score):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    font = ImageFont.truetype("arial", 400)
    draw = ImageDraw.Draw(dst)
    # .text(
    #     (0, 0),  # Coordinates
    #     'Hello world!',  # Text
    #     (0, 0, 0)  # Color
    # )
    draw.text((10, 10),str(score),(0,0,255),font=font)

    return dst

In [15]:
os.makedirs("./duplicated0")

In [16]:
count = 0
score_list = []
types = ["IMG","PANO"]
for idx in types:
    for k,v in tqdm(image_types[idx].items()):
        if(len(v) == 1):
            continue
        else:
            _check = list(combinations(v,2))
            for _img in _check:
                for i in range(3):
                    file = VOCROOTDIR + "/" + str(i) + "/" + _img[0]
                    if os.path.isfile(file):
                        imageA = cv2.imread(file)
                        break
                    else:
                        continue
                for i in range(3):
                    file = VOCROOTDIR + "/" + str(i) + "/" + _img[1]
                    if os.path.isfile(file):
                        imageB = cv2.imread(file)
                        break
                    else:
                        continue
                del file
                # print(_img)
                # print(imageA.shape, imageB.shape)
                if imageA.shape == imageB.shape:
                    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
                    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
                    (score, diff) = ssim(grayA, grayB, full=True)
                elif imageA.shape == (imageB.shape[1],imageB.shape[0]):
                    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

                    imageCCW = cv2.rotate(imageA, cv2.ROTATE_90_COUNTERCLOCKWISE)
                    grayCCW = cv2.cvtColor(imageCCW, cv2.COLOR_BGR2GRAY)
                    (scoreCCW, diffCCW) = ssim(grayCCW, grayB, full=True)

                    imageCW = cv2.rotate(imageA, cv2.ROTATE_90_CLOCKWISE)
                    grayCW = cv2.cvtColor(grayCW, cv2.COLOR_BGR2GRAY)
                    (scoreCW, diffCW) = ssim(grayCW, grayB, full=True)

                    if scoreCCW < scoreCW:
                        score = scoreCW
                        grayA = grayCW
                    else:
                        score = scoreCCW
                        grayA = grayCCW
                else:
                    continue
                score_list.append([_img, score])
                tmp = get_concat_h(to_pil_image(grayA),to_pil_image(grayB),score)
                # plt.imshow(tmp)
                # plt.show()
                plt.imshow(tmp); plt.axis('off'); plt.tight_layout(); #plt.show()
                plt.savefig('./duplicated0/%s.png'%(str(int(score*1000))+ "_" + _img[0][:-4] + "_" + _img[1][:-4]))
                plt.close('all')
                plt.clf()
                del tmp
            del _check


100%|██████████| 1/1 [00:00<?, ?it/s]


<Figure size 640x480 with 0 Axes>

In [27]:
len(sum(list(image_types['IMG'].values()),[]))
#label0 총 482쌍

482

In [17]:
for _img in _check:
    for i in range(3):
        file = VOCROOTDIR + "/" + str(i) + "/" + _img[0]
        if os.path.isfile(file):
            imageA = cv2.imread(file)
            break
        else:
            continue
    for i in range(3):
        file = VOCROOTDIR + "/" + str(i) + "/" + _img[1]
        if os.path.isfile(file):
            imageB = cv2.imread(file)
            break
        else:
            continue

NameError: name '_check' is not defined

In [28]:
count = 0
score_list = []
types = ["ETC"]
for idx in types:
    _check = list(combinations(image_types[idx],2))
    for _img in tqdm(_check):
        for i in range(3):
            file = VOCROOTDIR + "/" + str(i) + "/" + _img[0]
            if os.path.isfile(file):
                imageA = cv2.imread(file)
                break
            else:
                continue
        for i in range(3):
            file = VOCROOTDIR + "/" + str(i) + "/" + _img[1]
            if os.path.isfile(file):
                imageB = cv2.imread(file)
                break
            else:
                if i == 2:
                    print(file)
                continue
        del file
        # print(_img)
        # print(imageA.shape, imageB.shape)
        if imageA.shape == imageB.shape:
            grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
            grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
            (score, diff) = ssim(grayA, grayB, full=True)
        elif imageA.shape == (imageB.shape[1],imageB.shape[0]):
            grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

            imageCCW = cv2.rotate(imageA, cv2.ROTATE_90_COUNTERCLOCKWISE)
            grayCCW = cv2.cvtColor(imageCCW, cv2.COLOR_BGR2GRAY)
            (scoreCCW, diffCCW) = ssim(grayCCW, grayB, full=True)

            imageCW = cv2.rotate(imageA, cv2.ROTATE_90_CLOCKWISE)
            grayCW = cv2.cvtColor(grayCW, cv2.COLOR_BGR2GRAY)
            (scoreCW, diffCW) = ssim(grayCW, grayB, full=True)

            if scoreCCW < scoreCW:
                score = scoreCW
                grayA = grayCW
            else:
                score = scoreCCW
                grayA = grayCCW
        else:
            continue
        score_list.append([_img, score])
        tmp = get_concat_h(to_pil_image(grayA),to_pil_image(grayB),score)
        # plt.imshow(tmp)
        # plt.show()
        plt.imshow(tmp); plt.axis('off'); plt.tight_layout(); #plt.show()
        plt.savefig('./duplicated0/%s.png'%(str(int(score*1000))+ "_" + _img[0][:-4] + "_" + _img[1][:-4]))
        plt.close('all')
        plt.clf()
        del tmp
    del _check


100%|██████████| 91/91 [00:19<00:00,  4.72it/s]


<Figure size 640x480 with 0 Axes>

In [29]:
len(image_types[idx])

14

In [30]:
score_list.sort(key=lambda x: x[1])
score_list

[[('tmp_1571142043756.jpg', 'tmp_1575900337088.jpg'), 0.42531773372376086],
 [('1651193832-1.jpg', 'tmp_1635657899720.jpg'), 0.5239926264528089],
 [('1411450810-1.jpg', '1651193832-1.jpg'), 0.5440520161427577],
 [('1651193832-1.jpg', '290980310-1.jpg'), 0.565182741261053],
 [('1651193832-1.jpg', '2046329140-1.jpg'), 0.5851120132373681],
 [('1651193832-1.jpg', 'tmp_1621276119837.jpg'), 0.6008983506506894],
 [('1078694876-1.jpg', '1301837407-1.jpg'), 0.634205283730797],
 [('tmp_1621276119837.jpg', 'tmp_1635657899720.jpg'), 0.6572885549007222],
 [('290980310-1.jpg', 'tmp_1635657899720.jpg'), 0.6625858228907024],
 [('2046329140-1.jpg', 'tmp_1635657899720.jpg'), 0.6645271120419945],
 [('1078694876-1.jpg', '492669076-1.jpg'), 0.6757482842671656],
 [('1301837407-1.jpg', '492669076-1.jpg'), 0.6781640354037156],
 [('1078694876-1.jpg', '2077897036-1.jpg'), 0.6814405332802553],
 [('1078694876-1.jpg', '626443490-1.jpg'), 0.6871381516861607],
 [('1301837407-1.jpg', '2077897036-1.jpg'), 0.7043465474

---

## label 1

In [31]:
VOCROOTDIR = "../data/2022_10"
img_list = []
files = os.listdir(VOCROOTDIR + "/" + str(1) + "/" )
img_files = list(filter(lambda x: '.jpg' in x, files))
img_list += img_files

In [32]:
image_types = {"IMG":defaultdict(list), "PANO":defaultdict(list), "ETC":[]}

In [33]:
for file_name in img_list:
    _split = file_name.split("_")
    if 2 < len(_split):
        image_types[_split[0]][_split[1]].append(file_name)
    else:
        image_types["ETC"].append(file_name)

In [34]:
def get_concat_h(im1, im2,score):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    font = ImageFont.truetype("arial", 400)
    draw = ImageDraw.Draw(dst)
    # .text(
    #     (0, 0),  # Coordinates
    #     'Hello world!',  # Text
    #     (0, 0, 0)  # Color
    # )
    draw.text((10, 10),str(score),(0,0,255),font=font)

    return dst

In [35]:
from skimage.metrics import structural_similarity as ssim 
import argparse
import cv2
from skimage import io
from itertools import combinations
import matplotlib.pyplot as plt

from PIL import Image, ImageFont, ImageDraw

from torchvision.transforms.functional import to_pil_image
from tqdm import tqdm

In [36]:
os.makedirs("./duplicated1")

In [37]:
count = 0
score_list = []
types = ["IMG","PANO"]
for idx in types:
    for k,v in tqdm(image_types[idx].items()):
        if(len(v) == 1):
            continue
        else:
            _check = list(combinations(v,2))
            for _img in _check:
                for i in range(3):
                    file = VOCROOTDIR + "/" + str(i) + "/" + _img[0]
                    if os.path.isfile(file):
                        imageA = cv2.imread(file)
                        break
                    else:
                        continue
                for i in range(3):
                    file = VOCROOTDIR + "/" + str(i) + "/" + _img[1]
                    if os.path.isfile(file):
                        imageB = cv2.imread(file)
                        break
                    else:
                        continue
                del file
                # print(_img)
                # print(imageA.shape, imageB.shape)
                if imageA.shape == imageB.shape:
                    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
                    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
                    (score, diff) = ssim(grayA, grayB, full=True)
                elif imageA.shape == (imageB.shape[1],imageB.shape[0]):
                    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

                    imageCCW = cv2.rotate(imageA, cv2.ROTATE_90_COUNTERCLOCKWISE)
                    grayCCW = cv2.cvtColor(imageCCW, cv2.COLOR_BGR2GRAY)
                    (scoreCCW, diffCCW) = ssim(grayCCW, grayB, full=True)

                    imageCW = cv2.rotate(imageA, cv2.ROTATE_90_CLOCKWISE)
                    grayCW = cv2.cvtColor(grayCW, cv2.COLOR_BGR2GRAY)
                    (scoreCW, diffCW) = ssim(grayCW, grayB, full=True)

                    if scoreCCW < scoreCW:
                        score = scoreCW
                        grayA = grayCW
                    else:
                        score = scoreCCW
                        grayA = grayCCW
                else:
                    continue
                score_list.append([_img, score])
                tmp = get_concat_h(to_pil_image(grayA),to_pil_image(grayB),score)
                # plt.imshow(tmp)
                # plt.show()
                plt.imshow(tmp); plt.axis('off'); plt.tight_layout(); #plt.show()
                plt.savefig('./duplicated1/%s.png'%(str(int(score*1000))+ "_" + _img[0][:-4] + "_x_" + _img[1][:-4]))
                plt.close('all')
                plt.clf()
                del tmp
            del _check


100%|██████████| 152/152 [23:11<00:00,  9.16s/it]
0it [00:00, ?it/s]


<Figure size 640x480 with 0 Axes>

In [ ]:
for _img in _check:
    for i in range(3):
        file = VOCROOTDIR + "/" + str(i) + "/" + _img[0]
        if os.path.isfile(file):
            imageA = cv2.imread(file)
            break
        else:
            continue
    for i in range(3):
        file = VOCROOTDIR + "/" + str(i) + "/" + _img[1]
        if os.path.isfile(file):
            imageB = cv2.imread(file)
            break
        else:
            continue

In [40]:
count = 0
score_list = []
types = ["ETC"]
for idx in types:
    _check = list(combinations(image_types[idx],2))
    for _img in tqdm(_check):
        for i in range(3):
            file = VOCROOTDIR + "/" + str(i) + "/" + _img[0]
            if os.path.isfile(file):
                imageA = cv2.imread(file)
                break
            else:
                continue
        for i in range(3):
            file = VOCROOTDIR + "/" + str(i) + "/" + _img[1]
            if os.path.isfile(file):
                imageB = cv2.imread(file)
                break
            else:
                if i == 2:
                    print(file)
                continue
        del file
        # print(_img)
        # print(imageA.shape, imageB.shape)
        if imageA.shape == imageB.shape:
            grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
            grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
            (score, diff) = ssim(grayA, grayB, full=True)
        elif imageA.shape == (imageB.shape[1],imageB.shape[0]):
            grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

            imageCCW = cv2.rotate(imageA, cv2.ROTATE_90_COUNTERCLOCKWISE)
            grayCCW = cv2.cvtColor(imageCCW, cv2.COLOR_BGR2GRAY)
            (scoreCCW, diffCCW) = ssim(grayCCW, grayB, full=True)

            imageCW = cv2.rotate(imageA, cv2.ROTATE_90_CLOCKWISE)
            grayCW = cv2.cvtColor(grayCW, cv2.COLOR_BGR2GRAY)
            (scoreCW, diffCW) = ssim(grayCW, grayB, full=True)

            if scoreCCW < scoreCW:
                score = scoreCW
                grayA = grayCW
            else:
                score = scoreCCW
                grayA = grayCCW
        else:
            continue
        score_list.append([_img, score])
        tmp = get_concat_h(to_pil_image(grayA),to_pil_image(grayB),score)
        # plt.imshow(tmp)
        # plt.show()
        plt.imshow(tmp); plt.axis('off'); plt.tight_layout(); #plt.show()
        plt.savefig('./duplicated1/%s.png'%(str(int(score*1000))+ "_" + _img[0][:-4] + "_x_" + _img[1][:-4]))
        plt.close('all')
        plt.clf()
        del tmp
    del _check


100%|██████████| 3/3 [00:00<00:00, 12.10it/s]


<Figure size 640x480 with 0 Axes>

---
# label 2

In [41]:
VOCROOTDIR = "../data/2022_10"
img_list = []
files = os.listdir(VOCROOTDIR + "/" + str(2) + "/" )
img_files = list(filter(lambda x: '.jpg' in x, files))
img_list += img_files

In [42]:
image_types = {"IMG":defaultdict(list), "PANO":defaultdict(list), "ETC":[]}

In [43]:
for file_name in img_list:
    _split = file_name.split("_")
    if 2 < len(_split):
        image_types[_split[0]][_split[1]].append(file_name)
    else:
        image_types["ETC"].append(file_name)

In [44]:
def get_concat_h(im1, im2,score):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    font = ImageFont.truetype("arial", 400)
    draw = ImageDraw.Draw(dst)
    # .text(
    #     (0, 0),  # Coordinates
    #     'Hello world!',  # Text
    #     (0, 0, 0)  # Color
    # )
    draw.text((10, 10),str(score),(0,0,255),font=font)

    return dst

In [45]:
from skimage.metrics import structural_similarity as ssim 
import argparse
import cv2
from skimage import io
from itertools import combinations
import matplotlib.pyplot as plt

from PIL import Image, ImageFont, ImageDraw

from torchvision.transforms.functional import to_pil_image
from tqdm import tqdm

In [46]:
os.makedirs("./duplicated2")

In [47]:
count = 0
score_list = []
types = ["IMG","PANO"]
for idx in types:
    for k,v in tqdm(image_types[idx].items()):
        if(len(v) == 1):
            continue
        else:
            _check = list(combinations(v,2))
            for _img in _check:
                for i in range(3):
                    file = VOCROOTDIR + "/" + str(i) + "/" + _img[0]
                    if os.path.isfile(file):
                        imageA = cv2.imread(file)
                        break
                    else:
                        continue
                for i in range(3):
                    file = VOCROOTDIR + "/" + str(i) + "/" + _img[1]
                    if os.path.isfile(file):
                        imageB = cv2.imread(file)
                        break
                    else:
                        continue
                del file
                # print(_img)
                # print(imageA.shape, imageB.shape)
                if imageA.shape == imageB.shape:
                    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
                    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
                    (score, diff) = ssim(grayA, grayB, full=True)
                elif imageA.shape == (imageB.shape[1],imageB.shape[0]):
                    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

                    imageCCW = cv2.rotate(imageA, cv2.ROTATE_90_COUNTERCLOCKWISE)
                    grayCCW = cv2.cvtColor(imageCCW, cv2.COLOR_BGR2GRAY)
                    (scoreCCW, diffCCW) = ssim(grayCCW, grayB, full=True)

                    imageCW = cv2.rotate(imageA, cv2.ROTATE_90_CLOCKWISE)
                    grayCW = cv2.cvtColor(grayCW, cv2.COLOR_BGR2GRAY)
                    (scoreCW, diffCW) = ssim(grayCW, grayB, full=True)

                    if scoreCCW < scoreCW:
                        score = scoreCW
                        grayA = grayCW
                    else:
                        score = scoreCCW
                        grayA = grayCCW
                else:
                    continue
                score_list.append([_img, score])
                tmp = get_concat_h(to_pil_image(grayA),to_pil_image(grayB),score)
                # plt.imshow(tmp)
                # plt.show()
                plt.imshow(tmp); plt.axis('off'); plt.tight_layout(); #plt.show()
                plt.savefig('./duplicated2/%s.png'%(str(int(score*1000))+ "_" + _img[0][:-4] + "_x_" + _img[1][:-4]))
                plt.close('all')
                plt.clf()
                del tmp
            del _check


 82%|████████▏ | 142/174 [52:14<11:46, 22.08s/it]   


MemoryError: Unable to allocate 99.0 MiB for an array with shape (4160, 3120) and data type float64

<Figure size 640x480 with 0 Axes>

In [ ]:
for _img in _check:
    for i in range(3):
        file = VOCROOTDIR + "/" + str(i) + "/" + _img[0]
        if os.path.isfile(file):
            imageA = cv2.imread(file)
            break
        else:
            continue
    for i in range(3):
        file = VOCROOTDIR + "/" + str(i) + "/" + _img[1]
        if os.path.isfile(file):
            imageB = cv2.imread(file)
            break
        else:
            continue

In [ ]:
count = 0
score_list = []
types = ["ETC"]
for idx in types:
    _check = list(combinations(image_types[idx],2))
    for _img in tqdm(_check):
        for i in range(3):
            file = VOCROOTDIR + "/" + str(i) + "/" + _img[0]
            if os.path.isfile(file):
                imageA = cv2.imread(file)
                break
            else:
                continue
        for i in range(3):
            file = VOCROOTDIR + "/" + str(i) + "/" + _img[1]
            if os.path.isfile(file):
                imageB = cv2.imread(file)
                break
            else:
                if i == 2:
                    print(file)
                continue
        del file
        # print(_img)
        # print(imageA.shape, imageB.shape)
        if imageA.shape == imageB.shape:
            grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
            grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
            (score, diff) = ssim(grayA, grayB, full=True)
        elif imageA.shape == (imageB.shape[1],imageB.shape[0]):
            grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

            imageCCW = cv2.rotate(imageA, cv2.ROTATE_90_COUNTERCLOCKWISE)
            grayCCW = cv2.cvtColor(imageCCW, cv2.COLOR_BGR2GRAY)
            (scoreCCW, diffCCW) = ssim(grayCCW, grayB, full=True)

            imageCW = cv2.rotate(imageA, cv2.ROTATE_90_CLOCKWISE)
            grayCW = cv2.cvtColor(grayCW, cv2.COLOR_BGR2GRAY)
            (scoreCW, diffCW) = ssim(grayCW, grayB, full=True)

            if scoreCCW < scoreCW:
                score = scoreCW
                grayA = grayCW
            else:
                score = scoreCCW
                grayA = grayCCW
        else:
            continue
        score_list.append([_img, score])
        tmp = get_concat_h(to_pil_image(grayA),to_pil_image(grayB),score)
        # plt.imshow(tmp)
        # plt.show()
        plt.imshow(tmp); plt.axis('off'); plt.tight_layout(); #plt.show()
        plt.savefig('./duplicated2/%s.png'%(str(int(score*1000))+ "_" + _img[0][:-4] + "_x_" + _img[1][:-4]))
        plt.close('all')
        plt.clf()
        del tmp
    del _check


---

# 서브라벨 반영

In [39]:
import pandas as pd
import os

In [ ]:
label0_dups = glob("./duplicated0")
label1_dups = glob("./duplicated1")
label2_dups = glob("./duplicated2")

In [ ]:
for file_couples in label0_dups:
    first_file=file_couples.split("_x_")[0]
    second_file=file_couples.split("_x_")[1]
    first_file[]
    bmc_df.iloc[idx][['color','residue','turbidity']]